In [41]:
import pandas as pd
import datetime
import os
working_directory = os.getcwd()
print(working_directory)

/Users/vui_tinh/Desktop/Desktop_Stuff/Boston/CS_689/Project


In [3]:
path_1 = working_directory + '/listings_details_amsterdam.csv'
path_2 = working_directory + '/listings_detailed_madrid.csv'
path_3 = working_directory + '/AirBnB_Amsterdam/listings.csv'
path_4 = working_directory + '/AirBnB_Madrid/listings.csv'
path_5 = working_directory + '/AirBnB_Amsterdam/reviews_details.csv'
path_6 = working_directory + '/AirBnB_Madrid/reviews_detailed.csv'
path_7 = working_directory + '/AirBnB_Madrid/calendar.csv'
path_8 = working_directory + '/AirBnB_Amsterdam/calendar.csv'
Amsterdam_df = pd.read_csv(path_1)
Madrid_df  = pd.read_csv(path_2)
Amsterdam_listing_df = pd.read_csv(path_3)
Madrid_listing_df = pd.read_csv(path_4)
Amsterdam_review_df = pd.read_csv(path_5)
Madrid_review_df = pd.read_csv(path_6)
Madrid_Calendar = pd.read_csv(path_7)
Amsterdam_Calendar = pd.read_csv(path_8)

/var/folders/6c/xgblq5v97lb39fths3r32xvm0000gn/T/ipykernel_36194/2130609872.py:9: DtypeWarning: Columns (87) have mixed types. Specify dtype option on import or set low_memory=False.
  Amsterdam_df = pd.read_csv(path_1)


In [ ]:
#--1-------Doing for Host Dimension

In [5]:
# Filter columns for Amsterdam host dimension
Host_Amsterdam_df = Amsterdam_df.filter(['host_id','host_name','host_is_superhost','host_response_time'])
Host_Amsterdam_df

,host_id,host_name,host_is_superhost,host_response_time
0,3159,Daniel,t,within an hour
1,3806,Maartje,f,within an hour
2,59484,Alex,f,within a few hours
3,56142,Joan,f,within a few hours
4,97647,Flip,t,within an hour
...,...,...,...,...
20025,13399651,Marieke,f,NaN
20026,1595885,Evita,f,NaN
20027,87866499,Tommaso,f,NaN
20028,1595885,Evita,f,NaN


In [15]:
# Count combination records for host dimension
host_Amsterdam = Host_Amsterdam_df.groupby(['host_id','host_name','host_is_superhost','host_response_time'],dropna=False).size().reset_index(name='count')



In [16]:
# Check Duplicate
Duplicate_combination = host_Amsterdam[host_Amsterdam['count']>1]


In [17]:
# Drop duplicates for Host_Amsterdam and get distinct records
Host_Amsterdam_Dis_df = Host_Amsterdam_df.drop_duplicates()
Host_Amsterdam_Dis_df.head(5)

,host_id,host_name,host_is_superhost,host_response_time
0,3159,Daniel,t,within an hour
1,3806,Maartje,f,within an hour
2,59484,Alex,f,within a few hours
3,56142,Joan,f,within a few hours
4,97647,Flip,t,within an hour


In [18]:
# 2. Filter columns for Amsterdam host dimension
Host_Marid_df = Madrid_df.filter(['host_id','host_name','host_is_superhost','host_response_time'])


In [19]:
# Get duplicates records
host_Madrid = Host_Marid_df.groupby(['host_id','host_name','host_is_superhost','host_response_time'],dropna=False).size().reset_index(name='count')
Duplicate_combination2 = host_Madrid[host_Madrid['count']>1]



In [20]:
# Drop duplicates for Host_Madrid and get distinct records
Host_Madrid_Dis_df = Host_Marid_df.drop_duplicates()
Host_Madrid_Dis_df.head(5)

,host_id,host_name,host_is_superhost,host_response_time
0,13660,Simon,t,within an hour
1,83531,Abdel,f,NaN
2,82175,Jesus,f,within an hour
3,346366726,A,f,NaN
4,114340,Agustina,f,NaN


# ----------HOST_DIM-------Combine the two distinct data frame to get Host_dimension 

In [22]:
data = [Host_Amsterdam_Dis_df,Host_Madrid_Dis_df]
host_dim = pd.concat(data,ignore_index=True)
print(host_dim)

         host_id host_name host_is_superhost  host_response_time
0           3159    Daniel                 t      within an hour
1           3806   Maartje                 f      within an hour
2          59484      Alex                 f  within a few hours
3          56142      Joan                 f  within a few hours
4          97647      Flip                 t      within an hour
...          ...       ...               ...                 ...
28584  396428081     Maria                 f                 NaN
28585   78914217      Zami                 f  a few days or more
28586  151861190      Jose                 f                 NaN
28587   38544397   Juliane                 f  within a few hours
28588  269722123         Y                 f                 NaN

[28589 rows x 4 columns]


# Create Id for host_dim----------Host_dim ready to load in DBMS


In [29]:
host_dim['id'] = host_dim.reset_index().index+1
print(host_dim)

         host_id host_name host_is_superhost  host_response_time     id
0           3159    Daniel                 t      within an hour      1
1           3806   Maartje                 f      within an hour      2
2          59484      Alex                 f  within a few hours      3
3          56142      Joan                 f  within a few hours      4
4          97647      Flip                 t      within an hour      5
...          ...       ...               ...                 ...    ...
28584  396428081     Maria                 f                 NaN  28585
28585   78914217      Zami                 f  a few days or more  28586
28586  151861190      Jose                 f                 NaN  28587
28587   38544397   Juliane                 f  within a few hours  28588
28588  269722123         Y                 f                 NaN  28589

[28589 rows x 5 columns]


In [114]:
# check null values
host_dim.isna().sum()

host_id                   0
host_name               344
host_is_superhost       344
host_response_time    15591
id                        0
dtype: int64

In [116]:
# Convert null values to meaning values in this dataframe
host_dim['host_name'] = host_dim['host_name'].fillna('Anomynos')
host_dim['host_is_superhost'] = host_dim['host_is_superhost'].fillna('f') # logic is no means false
host_dim['host_response_time'] = host_dim['host_response_time'].fillna('no measures')

In [118]:
host_dim

,host_id,host_name,host_is_superhost,host_response_time,id
0,3159,Daniel,t,within an hour,1
1,3806,Maartje,f,within an hour,2
2,59484,Alex,f,within a few hours,3
3,56142,Joan,f,within a few hours,4
4,97647,Flip,t,within an hour,5
...,...,...,...,...,...
28584,396428081,Maria,f,no measures,28585
28585,78914217,Zami,f,a few days or more,28586
28586,151861190,Jose,f,no measures,28587
28587,38544397,Juliane,f,within a few hours,28588


# Host_Dim ready to load into DBMS

In [ ]:
host_dim

# --2-------Doing for Listing Dimension

In [30]:
# Filter for Amsterdam Listing Dimension (amsterdam listing staging table)
Amsterdam_listing = Amsterdam_df.filter(['id','name','property_type','room_type','price',
                                        'accommodates','amenities','description'])
Amsterdam_listing.head(5)

,id,name,property_type,room_type,price,accommodates,amenities,description
0,2818,Quiet Garden View Room & Super Fast WiFi,Apartment,Private room,$59.00,2,"{Internet,Wifi,""Paid parking off premises"",""Bu...",Quiet Garden View Room & Super Fast WiFi I'm r...
1,3209,"Quiet apt near center, great view",Apartment,Entire home/apt,$160.00,5,"{Internet,Wifi,Kitchen,""Paid parking off premi...",You will love our spacious (90 m2) bright apar...
2,20168,100%Centre-Studio 1 Private Floor/Bathroom,Townhouse,Entire home/apt,$80.00,2,"{TV,Internet,Wifi,""Paid parking off premises"",...","Cozy studio on your own private floor, 100% in..."
3,25428,Lovely apt in City Centre (Jordaan),Apartment,Entire home/apt,$125.00,3,"{TV,""Cable TV"",Internet,Wifi,Kitchen,Elevator,...","This nicely furnished, newly renovated apt is..."
4,27886,"Romantic, stylish B&B houseboat in canal district",Houseboat,Private room,$150.00,2,"{TV,Internet,Wifi,Breakfast,Heating,""Smoke det...",Stylish and romantic houseboat on fantastic hi...


In [31]:
# Drop Dupliates to get Amsterdam_listing distinct records
Amsterdam_listing_dist = Amsterdam_listing.drop_duplicates()

Amsterdam_listing_dist.head(5)

,id,name,property_type,room_type,price,accommodates,amenities,description
0,2818,Quiet Garden View Room & Super Fast WiFi,Apartment,Private room,$59.00,2,"{Internet,Wifi,""Paid parking off premises"",""Bu...",Quiet Garden View Room & Super Fast WiFi I'm r...
1,3209,"Quiet apt near center, great view",Apartment,Entire home/apt,$160.00,5,"{Internet,Wifi,Kitchen,""Paid parking off premi...",You will love our spacious (90 m2) bright apar...
2,20168,100%Centre-Studio 1 Private Floor/Bathroom,Townhouse,Entire home/apt,$80.00,2,"{TV,Internet,Wifi,""Paid parking off premises"",...","Cozy studio on your own private floor, 100% in..."
3,25428,Lovely apt in City Centre (Jordaan),Apartment,Entire home/apt,$125.00,3,"{TV,""Cable TV"",Internet,Wifi,Kitchen,Elevator,...","This nicely furnished, newly renovated apt is..."
4,27886,"Romantic, stylish B&B houseboat in canal district",Houseboat,Private room,$150.00,2,"{TV,Internet,Wifi,Breakfast,Heating,""Smoke det...",Stylish and romantic houseboat on fantastic hi...


In [33]:
# Filter for Madrid Listing Dimension (Madrid staging table)
Madrid_listing = Madrid_df.filter(['id','name','property_type','room_type','price',
                                   'accommodates','amenities','description'])
Madrid_listing.head(5)

,id,name,property_type,room_type,price,accommodates,amenities,description
0,6369,"Rooftop terrace room , ensuite bathroom",Private room in apartment,Private room,$60.00,2,"[""Kitchen"", ""Elevator"", ""Extra pillows and bla...",Excellent connection with the AIRPORT and EXHI...
1,21853,Bright and airy room,Private room in apartment,Private room,$31.00,1,"[""Bed linens"", ""Refrigerator"", ""Dishes and sil...",We have a quiet and sunny room with a good vie...
2,23001,Apartmento Arganzuela- Madrid Rio,Entire apartment,Entire home/apt,$50.00,6,"[""Pool"", ""Dishwasher"", ""Bed linens"", ""Refriger...","Apartamento de tres dormitorios dobles, gran s..."
3,24805,Gran Via Studio Madrid,Entire apartment,Entire home/apt,$92.00,3,"[""Refrigerator"", ""Host greets you"", ""Dishes an...","Studio located 50 meters from Gran Via, next t..."
4,26825,Single Room whith private Bathroom,Private room in house,Private room,$26.00,1,"[""Elevator"", ""Patio or balcony"", ""Bed linens"",...",Nice and cozy roon for one person with a priva...


In [35]:
# Drop Dupliates to get Amsterdam_listing distinct records
Madrid_listing_dist = Madrid_listing.drop_duplicates()

Madrid_listing_dist.head(5)

,id,name,property_type,room_type,price,accommodates,amenities,description
0,6369,"Rooftop terrace room , ensuite bathroom",Private room in apartment,Private room,$60.00,2,"[""Kitchen"", ""Elevator"", ""Extra pillows and bla...",Excellent connection with the AIRPORT and EXHI...
1,21853,Bright and airy room,Private room in apartment,Private room,$31.00,1,"[""Bed linens"", ""Refrigerator"", ""Dishes and sil...",We have a quiet and sunny room with a good vie...
2,23001,Apartmento Arganzuela- Madrid Rio,Entire apartment,Entire home/apt,$50.00,6,"[""Pool"", ""Dishwasher"", ""Bed linens"", ""Refriger...","Apartamento de tres dormitorios dobles, gran s..."
3,24805,Gran Via Studio Madrid,Entire apartment,Entire home/apt,$92.00,3,"[""Refrigerator"", ""Host greets you"", ""Dishes an...","Studio located 50 meters from Gran Via, next t..."
4,26825,Single Room whith private Bathroom,Private room in house,Private room,$26.00,1,"[""Elevator"", ""Patio or balcony"", ""Bed linens"",...",Nice and cozy roon for one person with a priva...


In [36]:
## ------LISTING-DIM--Combine the two distinct data frame (Amsterdam_listing_dist and Madrid_listing_dist)
data1 = [Amsterdam_listing_dist,Madrid_listing_dist]

Listing_Distinct = pd.concat(data1, ignore_index = True)

In [37]:
# Add effective_timestamp, Expire_timestamp and current_flag for listing_Dim
Listing_Distinct['effective_timestamp'] = pd.Timestamp.today().strftime('%Y-%m-%d')
Listing_Distinct['expire_timestamp'] = datetime.date(9999, 12, 31)
Listing_Distinct['current_flag'] = 'Current'
Listing_Distinct.head(2)

,id,name,property_type,room_type,price,accommodates,amenities,description,effective_timestamp,expire_timestamp,current_flag
0,2818,Quiet Garden View Room & Super Fast WiFi,Apartment,Private room,$59.00,2,"{Internet,Wifi,""Paid parking off premises"",""Bu...",Quiet Garden View Room & Super Fast WiFi I'm r...,2023-04-13,9999-12-31,Current
1,3209,"Quiet apt near center, great view",Apartment,Entire home/apt,$160.00,5,"{Internet,Wifi,Kitchen,""Paid parking off premi...",You will love our spacious (90 m2) bright apar...,2023-04-13,9999-12-31,Current


In [45]:
# Create listing dimension key
Listing_Distinct['listing_dim_id'] = Listing_Distinct.reset_index().index + 1

In [46]:
# Transform price in the listing_distinct
Listing_Distinct['price'] = Listing_Distinct['price'].str.extract('(\d+)').astype(int)

# ---This is listing dimension ready to load into dbms

In [48]:
Listing_Distinct.head(5) 

,id,name,property_type,room_type,price,accommodates,amenities,description,effective_timestamp,expire_timestamp,current_flag,listing_dim_id
0,2818,Quiet Garden View Room & Super Fast WiFi,Apartment,Private room,59,2,"{Internet,Wifi,""Paid parking off premises"",""Bu...",Quiet Garden View Room & Super Fast WiFi I'm r...,2023-04-13,9999-12-31,Current,1
1,3209,"Quiet apt near center, great view",Apartment,Entire home/apt,160,5,"{Internet,Wifi,Kitchen,""Paid parking off premi...",You will love our spacious (90 m2) bright apar...,2023-04-13,9999-12-31,Current,2
2,20168,100%Centre-Studio 1 Private Floor/Bathroom,Townhouse,Entire home/apt,80,2,"{TV,Internet,Wifi,""Paid parking off premises"",...","Cozy studio on your own private floor, 100% in...",2023-04-13,9999-12-31,Current,3
3,25428,Lovely apt in City Centre (Jordaan),Apartment,Entire home/apt,125,3,"{TV,""Cable TV"",Internet,Wifi,Kitchen,Elevator,...","This nicely furnished, newly renovated apt is...",2023-04-13,9999-12-31,Current,4
4,27886,"Romantic, stylish B&B houseboat in canal district",Houseboat,Private room,150,2,"{TV,Internet,Wifi,Breakfast,Heating,""Smoke det...",Stylish and romantic houseboat on fantastic hi...,2023-04-13,9999-12-31,Current,5


# --- Concate amsterdam and madrid listing staging tables

In [49]:

listing_data_df = [Amsterdam_df ,Amsterdam_df ]
Amsterdame_Madrid_staging = pd.concat(listing_data_df, ignore_index = True)

In [50]:
# Transform price
Amsterdame_Madrid_staging['price'] = Amsterdame_Madrid_staging['price'].str.extract('(\d+)').astype(int)

In [51]:
Amsterdame_Madrid_staging.shape

(40060, 96)

# ---3--------------Reviewer dimension

In [52]:

#--Amsterdam
Amsterdam_reviewer_df = Amsterdam_review_df.filter(['reviewer_id', 'reviewer_name','comments'])
Amsterdam_reviewer_dist = Amsterdam_reviewer_df.drop_duplicates()

In [53]:
Amsterdam_reviewer_dist.head(5)

,reviewer_id,reviewer_name,comments
0,10952,Lam,Daniel is really cool. The place was nice and ...
1,12798,Alice,Daniel is the most amazing host! His place is ...
2,11869,Natalja,We had such a great time in Amsterdam. Daniel ...
3,14064,Enrique,Very professional operation. Room is very clea...
4,17977,Sherwin,Daniel is highly recommended. He provided all...


In [54]:
#--Madrid
Madrid_reviewer_df = Madrid_review_df.filter(['reviewer_id', 'reviewer_name','comments'])
Madrid_reviewer_dist = Madrid_reviewer_df.drop_duplicates()
Madrid_reviewer_dist.head(5)

,reviewer_id,reviewer_name,comments
0,84790,Nancy,Simon and Arturo have the ultimate location in...
1,84338,David,Myself and Kristy originally planned on stayin...
2,98655,Marion,We had a great time at Arturo and Simon's ! A ...
3,109871,Kurt,I very much enjoyed the stay. \r\nIt's a wond...
4,98901,Dennis,Arturo and Simon are polite and friendly hosts...


,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2818,1191,2009-03-30,10952,Lam,Daniel is really cool. The place was nice and ...
1,2818,1771,2009-04-24,12798,Alice,Daniel is the most amazing host! His place is ...
2,2818,1989,2009-05-03,11869,Natalja,We had such a great time in Amsterdam. Daniel ...
3,2818,2797,2009-05-18,14064,Enrique,Very professional operation. Room is very clea...
4,2818,3151,2009-05-25,17977,Sherwin,Daniel is highly recommended. He provided all...


# ---Reviewer-Dim-----Combine Amsterdam_reviewer_dist and Madrid_reviewer_dist to get into Reviewer Dimension

In [62]:

data_reviewer = [Amsterdam_reviewer_dist,Madrid_reviewer_dist]
reviewer_dim = pd.concat(data_reviewer,ignore_index=True)
reviewer_dim.head(5)

,reviewer_id,reviewer_name,comments
0,10952,Lam,Daniel is really cool. The place was nice and ...
1,12798,Alice,Daniel is the most amazing host! His place is ...
2,11869,Natalja,We had such a great time in Amsterdam. Daniel ...
3,14064,Enrique,Very professional operation. Room is very clea...
4,17977,Sherwin,Daniel is highly recommended. He provided all...


In [64]:
# add new surrogate id as reviewer_dim_id
reviewer_dim['reviewer_dim_id'] = reviewer_dim.reset_index().index + 1

# This is reviewer dimension

In [65]:
reviewer_dim

,reviewer_id,reviewer_name,comments,reviewer_dim_id
0,10952,Lam,Daniel is really cool. The place was nice and ...,1
1,12798,Alice,Daniel is the most amazing host! His place is ...,2
2,11869,Natalja,We had such a great time in Amsterdam. Daniel ...,3
3,14064,Enrique,Very professional operation. Room is very clea...,4
4,17977,Sherwin,Daniel is highly recommended. He provided all...,5
...,...,...,...,...
1056261,133082364,Neus,"Ubicación genial, en el maravilloso barrio de ...",1056262
1056262,104936627,Irene,Muy buena atención y alojamiento muy cómodo.,1056263
1056263,196731765,Teodoro,muy buena comunicación y limpieza,1056264
1056264,266607018,Gerson,great spot great location jorge is very attentive,1056265


# ---Combine amsterdam_review_df and madrid_review_df as staging table

In [59]:

review_data_df = [Amsterdam_review_df ,Madrid_review_df ]
Amsterdame_Madrid_review_staging = pd.concat(review_data_df, ignore_index = True)

In [60]:
Amsterdame_Madrid_review_staging.head(5)

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2818,1191,2009-03-30,10952,Lam,Daniel is really cool. The place was nice and ...
1,2818,1771,2009-04-24,12798,Alice,Daniel is the most amazing host! His place is ...
2,2818,1989,2009-05-03,11869,Natalja,We had such a great time in Amsterdam. Daniel ...
3,2818,2797,2009-05-18,14064,Enrique,Very professional operation. Room is very clea...
4,2818,3151,2009-05-25,17977,Sherwin,Daniel is highly recommended. He provided all...


# Date Dimension

In [66]:

path_9 = working_directory + '/date_sample_k.csv'
dat_df = pd.read_csv(path_9)

In [67]:
path_10 = working_directory + '/2008-2013.csv'
dat_df2 = pd.read_csv(path_10)

In [69]:
path_11 = working_directory + '/2014-2019.csv'
dat_df3 = pd.read_csv(path_11)

In [70]:
frame = [dat_df2,dat_df3,dat_df]
date_staging = pd.concat(frame,ignore_index = True)

In [71]:
date_staging.columns

Index(['date key', 'full date', 'day of week', 'day num in month',
       'day num overall', 'day name', 'day abbrev', 'weekday flag',
       'week num in year', 'week num overall', 'week begin date',
       'week begin date key', 'month', 'month num overall', 'month name',
       'month abbrev', 'quarter', 'year', 'yearmo', 'fiscal month',
       'fiscal quarter', 'fiscal year', 'month end flag', 'same day year ago'],
      dtype='object')

In [72]:
date_dimension = date_staging.filter(['date key', 'full date', 'day num in month','month name','year']).drop_duplicates()
date_dimension_dist = date_dimension.rename(columns = {'date key':'time_id', 'full date': 'date', 
                                                       'day num in month':'day',
                                                       'month name': 'month'})

In [448]:
# Date_Dimension distinct
date_dimension_dist

,time_id,date,day,month,year
0,20080101,2008-01-01,1,January,2008
1,20080102,2008-01-02,2,January,2008
2,20080103,2008-01-03,3,January,2008
3,20080104,2008-01-04,4,January,2008
4,20080105,2008-01-05,5,January,2008
...,...,...,...,...,...
6570,20231227,2023-12-27,27,December,2023
6571,20231228,2023-12-28,28,December,2023
6572,20231229,2023-12-29,29,December,2023
6573,20231230,2023-12-30,30,December,2023


# 1.1 Listing Dimension

In [73]:

Listing_Distinct.shape

(39648, 12)

In [325]:
# Transform price in the Amsterdame_Madrid_staging
#Amsterdame_Madrid_staging['price'] = Amsterdame_Madrid_staging['price'].str.extract('(\d+)').astype(int)

# Join back listing dimension to amsterdam and madrid listing staging table

In [74]:

review_fact_1 = Amsterdame_Madrid_staging.merge(Listing_Distinct,how = 'left', on =
            ['id','name','property_type','room_type','price', 'accommodates','amenities','description'])


In [75]:
review_fact_1_listing = review_fact_1.rename(columns = {'id': 'listing_id'})


In [76]:
review_fact_1_listing.head(5)

,listing_id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,effective_timestamp,expire_timestamp,current_flag,listing_dim_id
0,2818,https://www.airbnb.com/rooms/2818,20181206172549,2018-12-06,Quiet Garden View Room & Super Fast WiFi,Quiet Garden View Room & Super Fast WiFi,I'm renting a bedroom (room overlooking the ga...,Quiet Garden View Room & Super Fast WiFi I'm r...,none,"Indische Buurt (""Indies Neighborhood"") is a ne...",...,f,strict_14_with_grace_period,f,f,1,2.10,2023-04-13,9999-12-31,Current,1
1,3209,https://www.airbnb.com/rooms/3209,20181206172549,2018-12-06,"Quiet apt near center, great view",You will love our spacious (90 m2) bright apar...,"Our apartment has lots of light, a balcony and...",You will love our spacious (90 m2) bright apar...,none,Welcome to the Spaarndammerbuurt! From the beg...,...,f,moderate,f,f,1,1.03,2023-04-13,9999-12-31,Current,2
2,20168,https://www.airbnb.com/rooms/20168,20181206172549,2018-12-06,100%Centre-Studio 1 Private Floor/Bathroom,"Cozy studio on your own private floor, 100% in...",For those who like all facets of city life. In...,"Cozy studio on your own private floor, 100% in...",none,Located just in between famous central canals....,...,f,strict_14_with_grace_period,f,f,2,2.18,2023-04-13,9999-12-31,Current,3
3,25428,https://www.airbnb.com/rooms/25428,20181206172549,2018-12-06,Lovely apt in City Centre (Jordaan),NaN,"This nicely furnished, newly renovated apt is...","This nicely furnished, newly renovated apt is...",none,NaN,...,f,strict_14_with_grace_period,f,f,2,0.09,2023-04-13,9999-12-31,Current,4
4,27886,https://www.airbnb.com/rooms/27886,20181206172549,2018-12-06,"Romantic, stylish B&B houseboat in canal district",Stylish and romantic houseboat on fantastic hi...,For a romantic couple: A beautifully restored ...,Stylish and romantic houseboat on fantastic hi...,none,"Central, quiet, safe, clean and beautiful.",...,f,strict_14_with_grace_period,f,f,1,2.03,2023-04-13,9999-12-31,Current,5


# --1.2 Reviewer dimension

In [80]:

reviewer_dim.head(5)

,reviewer_id,reviewer_name,comments,reviewer_dim_id
0,10952,Lam,Daniel is really cool. The place was nice and ...,1
1,12798,Alice,Daniel is the most amazing host! His place is ...,2
2,11869,Natalja,We had such a great time in Amsterdam. Daniel ...,3
3,14064,Enrique,Very professional operation. Room is very clea...,4
4,17977,Sherwin,Daniel is highly recommended. He provided all...,5


# join back to the review staging table

In [81]:

review_fact_2_r = Amsterdame_Madrid_review_staging.merge(reviewer_dim, how = 'inner', on = ['reviewer_id',
                                                                            'reviewer_name', 'comments'])

In [82]:
review_fact_2_r.head(5)

,listing_id,id,date,reviewer_id,reviewer_name,comments,reviewer_dim_id
0,2818,1191,2009-03-30,10952,Lam,Daniel is really cool. The place was nice and ...,1
1,2818,1771,2009-04-24,12798,Alice,Daniel is the most amazing host! His place is ...,2
2,2818,1989,2009-05-03,11869,Natalja,We had such a great time in Amsterdam. Daniel ...,3
3,2818,2797,2009-05-18,14064,Enrique,Very professional operation. Room is very clea...,4
4,2818,3151,2009-05-25,17977,Sherwin,Daniel is highly recommended. He provided all...,5


# -Date Dimension

In [85]:
date_dimension_dist.head(5)

,time_id,date,day,month,year
0,20080101,2008-01-01,1,January,2008
1,20080102,2008-01-02,2,January,2008
2,20080103,2008-01-03,3,January,2008
3,20080104,2008-01-04,4,January,2008
4,20080105,2008-01-05,5,January,2008


# Merge review_fact_2_r with date dimension

In [86]:

combine_review_fact_2_date = review_fact_2_r.merge(date_dimension_dist, how = 'inner', on = ['date'])


In [88]:
combine_review_fact_2_date.head(5)

,listing_id,id,date,reviewer_id,reviewer_name,comments,reviewer_dim_id,time_id,day,month,year
0,2818,1191,2009-03-30,10952,Lam,Daniel is really cool. The place was nice and ...,1,20090330,30,March,2009
1,2818,1771,2009-04-24,12798,Alice,Daniel is the most amazing host! His place is ...,2,20090424,24,April,2009
2,2818,1989,2009-05-03,11869,Natalja,We had such a great time in Amsterdam. Daniel ...,3,20090503,3,May,2009
3,2818,2797,2009-05-18,14064,Enrique,Very professional operation. Room is very clea...,4,20090518,18,May,2009
4,2818,3151,2009-05-25,17977,Sherwin,Daniel is highly recommended. He provided all...,5,20090525,25,May,2009


# ----Joining Amsterdam_Madrid Staging table, date dimension, review staging table

In [89]:
a = combine_review_fact_2_date.query("listing_id == 42713110") # some listing has reviewer review but does not have listing


In [92]:
a.head(2)

,listing_id,id,date,reviewer_id,reviewer_name,comments,reviewer_dim_id,time_id,day,month,year
842390,42713110,660870926,2020-09-06,267657260,Lidia,La habitación es idéntica a la foto. Buena loc...,1048969,20200906,6,September,2020
847560,42713110,659853171,2020-09-03,357225162,Francisco Jose,Perfecta habitación!,1048967,20200903,3,September,2020


In [93]:
b = review_fact_1_listing.query("listing_id ==1488680")
b

,listing_id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,effective_timestamp,expire_timestamp,current_flag,listing_dim_id


In [96]:
Review_fact_2 = combine_review_fact_2_date.merge(review_fact_1_listing, how = 'inner', on = 'listing_id')

In [554]:
####----Create Review_fact surrogate key
Review_fact_2['review_dim_id'] = Review_fact_2.reset_index().index + 1

In [97]:
Review_fact_2.head(5)

,listing_id,id,date,reviewer_id,reviewer_name,comments,reviewer_dim_id,time_id,day,month,...,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,effective_timestamp,expire_timestamp,current_flag,listing_dim_id
0,2818,1191,2009-03-30,10952,Lam,Daniel is really cool. The place was nice and ...,1,20090330,30,March,...,f,strict_14_with_grace_period,f,f,1,2.1,2023-04-13,9999-12-31,Current,1
1,2818,1191,2009-03-30,10952,Lam,Daniel is really cool. The place was nice and ...,1,20090330,30,March,...,f,strict_14_with_grace_period,f,f,1,2.1,2023-04-13,9999-12-31,Current,1
2,2818,1771,2009-04-24,12798,Alice,Daniel is the most amazing host! His place is ...,2,20090424,24,April,...,f,strict_14_with_grace_period,f,f,1,2.1,2023-04-13,9999-12-31,Current,1
3,2818,1771,2009-04-24,12798,Alice,Daniel is the most amazing host! His place is ...,2,20090424,24,April,...,f,strict_14_with_grace_period,f,f,1,2.1,2023-04-13,9999-12-31,Current,1
4,2818,1989,2009-05-03,11869,Natalja,We had such a great time in Amsterdam. Daniel ...,3,20090503,3,May,...,f,strict_14_with_grace_period,f,f,1,2.1,2023-04-13,9999-12-31,Current,1


In [ ]:
# Review Fact load data frame to load into DBMS

In [98]:

Review_fact_load_2 = Review_fact_2.filter(['review_dim_id','time_id','listing_dim_id','reviewer_dim_id',
                                     'review_scores_accuracy','review_scores_communication','review_scores_rating'])

In [105]:
Review_fact_load_2

,time_id,listing_dim_id,reviewer_dim_id,review_scores_accuracy,review_scores_communication,review_scores_rating
0,20090330,1,1,10.0,10.0,97.0
1,20090330,1,1,10.0,10.0,97.0
2,20090424,1,2,10.0,10.0,97.0
3,20090424,1,2,10.0,10.0,97.0
4,20090503,1,3,10.0,10.0,97.0
...,...,...,...,...,...,...
863665,20181206,20008,431758,8.0,10.0,100.0
863666,20160104,6274,249704,10.0,10.0,100.0
863667,20160104,6274,249704,10.0,10.0,100.0
863668,20180502,14722,398569,NaN,NaN,NaN


In [106]:
Review_fact_load_21 = Review_fact_load_2.fillna(0)

In [122]:
##### Loading data into dbms
import psycopg2 as pg
conn = pg.connect(user = "postgres",
                 password = "BU669",
                 host = "127.0.0.1",
                 port = "5434",
                 database = "CS689_Final_project")
cur = conn.cursor()


In [418]:
# Create Reviewer_dim table in database
cur.execute("""CREATE TABLE IF NOT EXISTS Reviewer_dim(reviewer_id int Primary Key, reviewer_name varchar,
                comments varchar);""")
conn.commit()


In [424]:
# Load Reviewr_dim data into DBMS
for index, row in reviewer_dim.iterrows():
    cur.execute("INSERT INTO Reviewer_dim(reviewer_id, reviewer_name, comments) values (%s,%s,%s);",
               (row.reviewer_dim_id, row.reviewer_name, row.comments))
conn.commit()

In [453]:
# Create Time_dim table in database
cur.execute("""CREATE TABLE IF NOT EXISTS Time_dim(time_id int Primary Key, date date, 
                day int, month varchar, year int);""")
conn.commit()


In [455]:
# Load time_dim data into DBMS
for index, row in date_dimension_dist.iterrows():
    cur.execute("INSERT INTO time_dim(time_id, date, day, month, year) values (%s,%s,%s,%s,%s);",
               (row.time_id, row.date, row.day, row.month, row.year))
conn.commit()

In [493]:
# Create Listing_dim table in database
cur.execute("""CREATE TABLE IF NOT EXISTS Listing_dim(listing_dim_id int Primary Key, listing_id int, name varchar,
                property_type varchar, room_type varchar, price int, accommodates varchar, amenities varchar,
                description varchar, effective_timestamp date, expire_timestamp date, current_flag varchar);""")
conn.commit()


In [582]:
# Load Listing_dim data into DBMS
for index, row in Listing_Distinct_load.iterrows():
    cur.execute("""INSERT INTO Listing_dim(listing_dim_id, listing_id,  name, property_type,
                room_type, price, accommodates, amenities, description, effective_timestamp, 
                expire_timestamp, current_flag ) values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);""",
               (row.listing_dim_id, row.id, row.title, row.property_type, row.room_type, row.price,
                row.accommodates, row.amenities, row.description, row.effective_timestamp, 
                row.expire_timestamp, row.current_flag))
conn.commit()

In [494]:
### Dataframe for loading into Listing_dim
Listing_Distinct_load = Listing_Distinct.rename(columns = {'name' : 'title'})


In [586]:
# # Create Review_fact table in database
cur.execute("""CREATE TABLE IF NOT EXISTS Review_fact(review_id bigint Primary Key, 
                time_id bigint References Time_dim (time_id), listing_id bigint References Listing_dim(listing_dim_id), 
                reviewer_id bigint References reviewer_dim (reviewer_id), 
                reviews_scores_accuracy bigint, review_scores_communication bigint, review_score_rating bigint);""")
conn.commit()


In [593]:
# Load Review_fact data into DBMS
for index, row in Review_fact_load_21.iterrows():
    listing_dim_id = int(row.listing_dim_id) if not pd.isna(row.listing_dim_id) else None
    review_scores_accuracy = int(row.review_scores_accuracy) if not pd.isna(row.review_scores_accuracy) else None
    review_scores_communication = int(row.review_scores_communication) if not pd.isna(row.review_scores_communication) else None
    review_scores_rating = int(row.review_scores_rating) if not pd.isna(row.review_scores_rating) else None
    time_id = int(row.time_id) if not pd.isna(row.time_id) else None
    reviewer_dim_id = int(row.reviewer_dim_id) if not pd.isna(row.reviewer_dim_id) else None
    cur.execute("""INSERT INTO Review_fact(review_id, time_id, listing_id, reviewer_id, reviews_scores_accuracy,
                review_scores_communication, review_score_rating) values (%s,%s,%s,%s,%s,%s,%s);""",
               (row.review_dim_id, time_id, listing_dim_id, reviewer_dim_id, review_scores_accuracy,
                review_scores_communication,row.review_scores_rating))
conn.commit()


In [123]:
# Load host_dim data into DBMS. The host_dim table was already created in DBMS with host_id serial primary key,
# host_name,host_is_superhost, host_response_time 
for index, row in host_dim.iterrows():
    cur.execute("""INSERT INTO host_dim(host_name, host_is_superhost, host_response_time)
                values(%s,%s,%s)""",
                (row.host_name, row.host_is_superhost, row.host_response_time))
conn.commit()    

In [588]:
# create new

In [124]:
host_dim

,host_id,host_name,host_is_superhost,host_response_time,id
0,3159,Daniel,t,within an hour,1
1,3806,Maartje,f,within an hour,2
2,59484,Alex,f,within a few hours,3
3,56142,Joan,f,within a few hours,4
4,97647,Flip,t,within an hour,5
...,...,...,...,...,...
28584,396428081,Maria,f,no measures,28585
28585,78914217,Zami,f,a few days or more,28586
28586,151861190,Jose,f,no measures,28587
28587,38544397,Juliane,f,within a few hours,28588
